In [122]:
import os
import geopandas as gpd
from pprint import pprint
import numpy as np
import pandas as pd
import sodapy as sp
import pandas_bokeh as pbk
pbk.output_notebook()
from sodapy import Socrata
import matplotlib.pyplot as plt
import geopandas as gpd
import descartes 
from bokeh.plotting import figure, output_file, show

Loading BokehJS ...

In [126]:
#Load data
cp_spain_administr = gpd.read_file("data/BCN500_0101S_LIMITE_ADM.shp", crs="epsg:4258")# streets spain
cp_spain_mainstreets  = gpd.read_file("data/BCN500_0602L_CARRETERA_PPAL.shp", crs="epsg:4258")# streets spain, csr 
#according to documentation of the dataset
cp_spain_smallstreets = gpd.read_file("data/BCN500_0601L_AUTOP_AUTOV.shp", crs="epsg:4258")
provincias_catalunya = cp_spain_administr[cp_spain_administr["CCAA"]=="Cataluña"]
streets_in_catalunya_main =  gpd.read_file("data/streets_in_catalunya_main.shp", crs="epsg:4258")
streets_in_catalunya_small = gpd.read_file("data/streets_in_catalunya_small.shp", crs="epsg:4258")
streets_in_catalunya_main["num_TIPO_0602"] = streets_in_catalunya_main["TIPO_0602"].astype(float)
streets_in_catalunya_small["num_TIPO_0601"] = streets_in_catalunya_small["TIPO_0601"].astype(float)
# Meteorological stations
client = Socrata("analisi.transparenciacatalunya.cat", None)
stations = client.get("yqwd-vj5e", limit=2000)
# Convert to pandas DataFrame
stations_df = pd.DataFrame.from_records(stations)
stations_gdf = gpd.GeoDataFrame(stations_df, geometry=gpd.points_from_xy(stations_df.longitud, stations_df.latitud))
stations_gdf.crs = 'epsg:4326' # the database page indicates projection "WSG84", which corresponds to crs 'epsg:4326'
stations_gdf = stations_gdf.to_crs("epsg:4258")
df_cities = pd.DataFrame({'City': ['Buenos Aires', 'Brasilia', 'Santiago', 'Bogota', 'Caracas'],'Country': ['Argentina', 'Brazil', 'Chile', 'Colombia', 'Venezuela'],'Latitude': [-34.58, -15.78, -33.45, 4.60, 10.48],'Longitude': [-58.66, -47.91, -70.66, -74.08, -66.86]})
gdf_cities = gpd.GeoDataFrame(df_cities, geometry=gpd.points_from_xy(df_cities.Longitude, df_cities.Latitude))
gdf_cities.crs='epsg:4258'
gdf_cities = gdf_cities.to_crs("epsg:4258")
df_prov = pd.DataFrame({'City': ['Barcelona', 'Tarragona', 'Girona', 'Lleida'],'Latitude': [41.73, 41.08, 42.128, 42.04],'Longitude': [1.984, 0.818, 2.6735, 1.0479]})
gdf_prov = gpd.GeoDataFrame(df_prov, geometry=gpd.points_from_xy(df_prov.Longitude, df_prov.Latitude))
gdf_prov.crs='epsg:4258'
gdf_prov = gdf_prov.to_crs("epsg:4258")
mcp = gpd.read_file('data/divisions-administratives-v1r0-comarques-1000000-20210122.shp', crs='epsg:25831')
cols = mcp.select_dtypes(include=[np.object]).columns
mcp[cols] = mcp[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))

client = Socrata("analisi.transparenciacatalunya.cat", None)
results = client.get("rmgc-ncpb", limit=16774)
# Convert to pandas DataFrame
results_df =  gpd.GeoDataFrame.from_records(results)

df_pobl = pd.read_excel("PoblacioComarques.xls")

# Ara fem la sel·lecció de provincies a representar en el mapa Bokeh

com = pd.value_counts(results_df['nomcom'])
com = com.to_frame() # convertir a Data Frame
com = com.reset_index() # introduir un index (0, 1, 2...)
com = com.sort_values("index") # la columna de velocitat es diu index, ordenar-la
com = com.rename(columns={"index": "nomcom", "nomcom": "#accidents"}) # canviar els noms correctes
Comar = gpd.GeoDataFrame(com).set_index("nomcom")
Comar = Comar.reset_index()

prov = results_df[['nomcom', 'nomdem']]
prov = prov.sort_values('nomcom') 

merge = prov.merge(Comar, left_on='nomcom', right_on='nomcom') # associem la comarca a la província, merge 16 mil
#filtrar Bcn
merge.set_index('nomdem', inplace=True)
merge_bcn=merge.loc[['Barcelona']]
#eliminar duplicats
merge_bcn = merge_bcn.drop_duplicates(subset='nomcom',keep='first')
bcn=merge_bcn.reset_index()

v_tgn=[1,2,3,4,5,6,9,10,12,13,14,16,17,18,19,20,22,23,24,25,26,27,30,31,32,33,34,37,38,39,40,41]
Tarragona = np.array(v_tgn)
v_bcn=[0,1,3,4,7,8,9,11,14,15,17,18,19,21,22,24,25,26,27,28,29,30,31,32,34,35,36,37,38]
Barcelona = np.array(v_bcn)
v_lld=[0,1,2,5,6,7,8,9,10,11,12,15,16,18,19,20,21,23,27,28,29,30,33,35,36,39,40,41]
Lleida = np.array(v_lld)
v_gir=[0,2,3,4,5,6,7,8,10,11,12,13,15,16,17,20,21,22,24,25,26,28,29,31,32,34,35,36,37,38,39,40,41]
Girona = np.array(v_gir)
mcp = mcp.drop(Barcelona)

geom = mcp[["NOMCOMAR", "geometry"]] # escollir les columnes: comarca i geometria de mcp
geom = geom.sort_values('NOMCOMAR') # ordenar alfabèticament
Geom = gpd.GeoDataFrame(geom).set_index("NOMCOMAR") # tornem a posar l'índex
Geom = Geom.reset_index()
Geom = Geom.rename(columns={"NOMCOMAR": "nomcom"}) # canviar el nom

merge2 = prov.merge(Geom, left_on='nomcom', right_on='nomcom') # associem la comarca a la província
#filtrar Bcn
merge2.set_index('nomdem', inplace=True)
merge2_bcn=merge.loc[['Barcelona']]
#eliminar duplicats
merge2_bcn = merge2_bcn.drop_duplicates(subset='nomcom',keep='first')
bcn2=merge_bcn.reset_index()

merge_norm = prov.merge(df_pobl, left_on='nomcom', right_on='comarca') # associem la comarca a la província
#filtrar Bcn
merge_norm.set_index('nomdem', inplace=True)
merge_bcn_norm=merge_norm.loc[['Barcelona']]
#eliminar duplicats
merge_bcn_norm = merge_bcn_norm.drop_duplicates(subset='nomcom',keep='first')
bcn_norm = merge_bcn_norm.reset_index()

norm = bcn_norm[['pobl', 'pobl_percent']]
combo = pd.concat([bcn, Geom, norm], axis=1) # juntem comarques, #accidents, geometria i poblacions de bcn
combo['accpercadamilhab'] = combo['#accidents']/combo['pobl']*1000 # afegim col per a #accidents per cada 1000 habitants
# combo = combo[['#accidents', 'nomcom', 'geometry', 'accpercadamilhab']]
combo=combo.dropna()
# combo

combo =  gpd.GeoDataFrame.from_records(combo)
combo.crs = "epsg:25831"
p = combo.to_crs("epsg:4258").plot_bokeh(legend="Comarcas with car accident density", category='accpercadamilhab',show_figure=False)

show(p)

<ipython-input-126-b62d235d6713>:28: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cols = mcp.select_dtypes(include=[np.object]).columns
C:\Users\Daniel\anaconda3\lib\site-packages\pandas_bokeh\geoplot.py:144: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for polygon in geometry:
